In [3]:
# Dependencies
import os
import pathlib
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from uszipcode import SearchEngine


# Import the API key
from config import geoapify_key
from config import census_key


/Users/benrichardson/anaconda3/envs/PythonDataYML/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


DATA CLEANUP

1. The first set of data was obtained from the OpenBreweryAPI. The API is a free API, however it limits the number of queries to 200 rows and restricting sourcing of data to 2 pages at a time. In order to source all necessary data, it was necessary to create a loop to move through all 156 pages of the API and collect 100 cases per page or 200 every two pages. 

In [4]:
# URL for GET requests to retrieve brewery data
base_url = 'https://api.openbrewerydb.org/v1/breweries?by_country=United_States'


# Define an empty list to fetch the page data for each set of data
brewery_data = []

# Loop through the pages to get all the data
for page in range(1, 156):
    query_url = base_url + "&page="+ str(page) + "&per_page=200"
    response = requests.get(query_url).json()
    brewery_data.append(response)

# Create a list of all the breweries
breweries = []
for page in brewery_data:
    for brewery in page:
        breweries.append(brewery)

# Create a dataframe from the list of breweries
breweries_df = pd.DataFrame(breweries)

breweries_df.head()


,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street
0,5128df48-79fc-4f0f-8b52-d06be54d0cec,(405) Brewing Co,micro,1716 Topeka St,None,None,Norman,Oklahoma,73069-8224,United States,-97.46818222,35.25738891,4058160490,http://www.405brewing.com,Oklahoma,1716 Topeka St
1,9c5a66c8-cc13-416f-a5d9-0a769c87d318,(512) Brewing Co,micro,407 Radam Ln Ste F200,None,None,Austin,Texas,78745-1197,United States,None,None,5129211545,http://www.512brewing.com,Texas,407 Radam Ln Ste F200
2,ef970757-fe42-416f-931d-722451f1f59c,10 Barrel Brewing Co,large,1501 E St,None,None,San Diego,California,92101-6618,United States,-117.129593,32.714813,6195782311,http://10barrel.com,California,1501 E St
3,6d14b220-8926-4521-8d19-b98a2d6ec3db,10 Barrel Brewing Co,large,62970 18th St,None,None,Bend,Oregon,97701-9847,United States,-121.281706,44.08683531,5415851007,http://www.10barrel.com,Oregon,62970 18th St
4,e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,None,None,Bend,Oregon,97703-2465,United States,-121.3288021,44.0575649,5415851007,None,Oregon,1135 NW Galveston Ave Ste B


It can be seen right away that the dataframe might be missing some values - the most importantly latitude and longitue which was important for maping visualizations. 

The describe function below shows that 2,392 rows are misisng the longitude and latitude. Since the number of rows consitutde 30% of the full data set, 
it was important to backfull the missing information.   

In [80]:
#creates a summary table of the data
breweries_df.describe()

,id,name,brewery_type,address_1,address_2,address_3,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street
count,7936,7936,7936,7153,4,0,7936,7936,7936,7936,5544,5544,7141,6787,7936,7153
unique,7936,7791,10,7064,4,0,2908,54,7695,2,5433,5433,6950,6437,54,7064
top,5128df48-79fc-4f0f-8b52-d06be54d0cec,Granite City Food & Brewery,micro,303 Main St,Estacada,NaN,Portland,California,64108,United States,-112.0773456,33.4485866,5122442739,http://www.gcfb.net,California,303 Main St
freq,1,6,4115,3,1,NaN,105,912,5,7935,6,6,5,23,912,3


In [5]:
#Prepare the breweries Database for analysis 
#drop brevery id, address_2, address_3, country, phone, website_url, and street
clean_breweries_df = breweries_df.drop(columns=['id', 'address_2', 'address_3', 'country', 'phone', 'website_url', 'street'])
clean_breweries_df.head()

,name,brewery_type,address_1,city,state_province,postal_code,longitude,latitude,state
0,(405) Brewing Co,micro,1716 Topeka St,Norman,Oklahoma,73069-8224,-97.46818222,35.25738891,Oklahoma
1,(512) Brewing Co,micro,407 Radam Ln Ste F200,Austin,Texas,78745-1197,None,None,Texas
2,10 Barrel Brewing Co,large,1501 E St,San Diego,California,92101-6618,-117.129593,32.714813,California
3,10 Barrel Brewing Co,large,62970 18th St,Bend,Oregon,97701-9847,-121.281706,44.08683531,Oregon
4,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,Bend,Oregon,97703-2465,-121.3288021,44.0575649,Oregon


Here's further confirmation that the necessary longtidue and latitude info is misisng in large numbers.   

In [6]:
#count missing values for each column
clean_breweries_df.isnull()
#count missing values for each column 
clean_breweries_df.isnull().sum()

# TEAM: We need to decide how to handle the missing values.  Do we drop the rows?

name                 0
brewery_type         0
address_1          783
city                 0
state_province       0
postal_code          0
longitude         2392
latitude          2392
state                0
dtype: int64

Decided to use Geopify to pull latitude and longtiude for the missing zip codes. Since Geopify did not recognize the postal code +4 format; 
opted to use the uszipcode package to extract the first 5 digits. While long/lat can be also pulled via uszipcode package, the exercise was to demonstrate the use of API for sourcing additional information. Using the package will also allow pulling additional information later.  

In [7]:
#create a new column in dataframe with first 5 digits of zip code ensuring the information is stored as stings & recognized as zipcodes.

search = SearchEngine()

def extract_zipcode(zipcode):
    # Remove +4 extension if present
    zipcode = zipcode.split('-')[0]
    zipcode_obj = search.by_zipcode(zipcode)
    if zipcode_obj is not None:
        return str(zipcode_obj.zipcode)[:5].zfill(5)
    else:
        return None

clean_breweries_df['zip_code'] = clean_breweries_df['postal_code'].apply(extract_zipcode).astype(str)

#add a ditinct ID number to each row 
clean_breweries_df['brewery_id'] = clean_breweries_df.index + 1
clean_breweries_df.head()


,name,brewery_type,address_1,city,state_province,postal_code,longitude,latitude,state,zip_code,brewery_id
0,(405) Brewing Co,micro,1716 Topeka St,Norman,Oklahoma,73069-8224,-97.46818222,35.25738891,Oklahoma,73069,1
1,(512) Brewing Co,micro,407 Radam Ln Ste F200,Austin,Texas,78745-1197,None,None,Texas,78745,2
2,10 Barrel Brewing Co,large,1501 E St,San Diego,California,92101-6618,-117.129593,32.714813,California,92101,3
3,10 Barrel Brewing Co,large,62970 18th St,Bend,Oregon,97701-9847,-121.281706,44.08683531,Oregon,97701,4
4,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,Bend,Oregon,97703-2465,-121.3288021,44.0575649,Oregon,97703,5


In [8]:

#load dataframe into csv
clean_breweries_df.to_csv('csv building blocks/breweries.csv', index=False)
#show count by the length of the postal code to see if there are any invalid postal codes
clean_breweries_df['zip_code'].str.len().value_counts()

5    7929
4       7
Name: zip_code, dtype: int64

In [9]:
#use dataframe from the csv breveries.csv
input_path = os.path.join("csv building blocks/breweries.csv")
csv_breweries_df = pd.read_csv(input_path)

In [12]:
#Quick test if the API request url worked for a specific zip code
#postcode = "78745"
#target_url = f"https://api.geoapify.com/v1/geocode/autocomplete?text={postcode}&limit=1&type=postcode&format=json&apiKey={geoapify_key}"
#response = requests.get(target_url).json()
#print(json.dumps(response, indent=4, sort_keys=True))


Creating a dataframe with missing information to temporarily separate from the clean of the data.

In [10]:
#extract rows with missing longitude and create a new dataframe - all longitude missing values correspond to missinging latitude values
missing_long_df = csv_breweries_df[csv_breweries_df['longitude'].isnull()]

#create a new dataframe with only the missing longitude rowsbased on clean_breweries_df
missing_long_df = clean_breweries_df[clean_breweries_df['longitude'].isnull()]

missing_long_df.head()


,name,brewery_type,address_1,city,state_province,postal_code,longitude,latitude,state,zip_code,brewery_id
1,(512) Brewing Co,micro,407 Radam Ln Ste F200,Austin,Texas,78745-1197,None,None,Texas,78745,2
18,12 Gates Brewing Company,brewpub,80 Earhart Dr Ste 20,Williamsville,New York,14221-7804,None,None,New York,14221,19
19,12 West Brewing Company,micro,3000 E Ray Rd Bldg 6,Gilbert,Arizona,85296-7832,None,None,Arizona,85296,20
24,12welve Eyes Brewing,micro,141 E 4th St Ste LL2,Saint Paul,Minnesota,55101-1639,None,None,Minnesota,55101,25
26,13 Stripes Brewery,brewpub,"250 Mill St, Suite PW3101",Taylors,South Carolina,29687,None,None,South Carolina,29687,27


In [11]:
#drop the missing_long dataframe into a csv file

missing_long_df.to_csv('csv building blocks/breweries_missing_values.csv', index=False)


In [12]:

#use dataframe from the csv_breweries_missing_values.csv
input_path2 = os.path.join("csv building blocks/breweries_missing_values.csv")
csv_breweries_missing_df = pd.read_csv(input_path2)


Finally demonstrate use of Geopify for obtianing missing information - encountered errors were bypassed with the try/except. 

In [89]:
#for missing longitude, use geopify to look up the longitude and latitude
for index, row in csv_breweries_missing_df.iterrows():
    postcode = row['zip_code']
    target_url = f"https://api.geoapify.com/v1/geocode/autocomplete?text={postcode}&limit=1&type=postcode&format=json&apiKey={geoapify_key}"
    response = requests.get(target_url)

    try:
        json_response = json.loads(response.text)
        if json_response:
            csv_breweries_missing_df.loc[index, 'longitude'] = json_response['results'][0]['lon']
            csv_breweries_missing_df.loc[index, 'latitude'] = json_response['results'][0]['lat']
            print(f"index: {index}, postcode: {postcode}, longitude: {json_response['results'][0]['lon']}, latitude: {json_response['results'][0]['lat']}") 

        else:
            print(f"Error: Empty JSON response for index {index}, postcode {postcode}")
    except IndexError:
        print(f"Error: IndexError occurred for index {index}, postcode {postcode}")
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON response for index {index}, postcode {postcode}")

csv_breweries_missing_df.head()

index: 0, postcode: 78745, longitude: -97.792614846, latitude: 30.208605656
index: 1, postcode: 14221, longitude: -78.729920877, latitude: 42.980952841
index: 2, postcode: 85296, longitude: -111.762518433, latitude: 33.335136701
index: 3, postcode: 55101, longitude: -93.088300242, latitude: 44.955919207
index: 4, postcode: 29687, longitude: -82.327800669, latitude: 34.991467109
index: 5, postcode: 27603, longitude: -78.66089377, latitude: 35.71301003
index: 6, postcode: 56442, longitude: -94.116315338, latitude: 46.677860236
index: 7, postcode: 05478, longitude: -73.104390267, latitude: 44.803484605
index: 8, postcode: 21157, longitude: -76.983937143, latitude: 39.558197671
index: 9, postcode: 22553, longitude: -77.60125509, latitude: 38.231176411
index: 10, postcode: 11741, longitude: -73.070358674, latitude: 40.794962148
index: 11, postcode: 05452, longitude: -73.08637315, latitude: 44.51595767
index: 12, postcode: 20109, longitude: -77.506105128, latitude: 38.784153209
index: 13, po

In [13]:
#save the dataframe with filled info into a csv file csv_breweries_missing_completed_df
csv_breweries_missing_df.to_csv('csv building blocks/breweries_missing_completed.csv', index=False)
csv_breweries_missing_df.head()

,name,brewery_type,address_1,city,state_province,postal_code,longitude,latitude,state,zip_code,brewery_id
0,(512) Brewing Co,micro,407 Radam Ln Ste F200,Austin,Texas,78745-1197,NaN,NaN,Texas,78745,2
1,12 Gates Brewing Company,brewpub,80 Earhart Dr Ste 20,Williamsville,New York,14221-7804,NaN,NaN,New York,14221,19
2,12 West Brewing Company,micro,3000 E Ray Rd Bldg 6,Gilbert,Arizona,85296-7832,NaN,NaN,Arizona,85296,20
3,12welve Eyes Brewing,micro,141 E 4th St Ste LL2,Saint Paul,Minnesota,55101-1639,NaN,NaN,Minnesota,55101,25
4,13 Stripes Brewery,brewpub,"250 Mill St, Suite PW3101",Taylors,South Carolina,29687,NaN,NaN,South Carolina,29687,27


In [14]:
#Andrews dataframe merge --> complete breweries dataframe
# Drop rows with blank long/lat values from total set of breweries 
breweries_dropped_nan = csv_breweries_df.dropna(subset=["longitude", "latitude"])

# Output list of breweries with no long/lat blanks to csv to combine with missing data 
breweries_dropped_nan.to_csv("csv building blocks/breweries_dropped.csv", index=False)
breweries_dropped_nan.head()

,name,brewery_type,address_1,city,state_province,postal_code,longitude,latitude,state,zip_code,brewery_id
0,(405) Brewing Co,micro,1716 Topeka St,Norman,Oklahoma,73069-8224,-97.468182,35.257389,Oklahoma,73069,1
2,10 Barrel Brewing Co,large,1501 E St,San Diego,California,92101-6618,-117.129593,32.714813,California,92101,3
3,10 Barrel Brewing Co,large,62970 18th St,Bend,Oregon,97701-9847,-121.281706,44.086835,Oregon,97701,4
4,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,Bend,Oregon,97703-2465,-121.328802,44.057565,Oregon,97703,5
5,10 Barrel Brewing Co,large,1411 NW Flanders St,Portland,Oregon,97209-2620,-122.685506,45.525979,Oregon,97209,6


In [15]:
# Read in csv files to combine 
df1 = pd.read_csv("csv building blocks/breweries_dropped.csv")
df2 = pd.read_csv("csv building blocks/breweries_missing_completed.csv")

# Combine csv files 
df_combined = pd.concat([df1, df2])

# Four rows with blank long/lat values still appearing; drop those rows 
complete_breweries = df_combined.dropna(subset=["longitude", "latitude"])

# Output list of breweries with complete long/lat values 
complete_breweries.to_csv("csv building blocks/breweries_complete.csv", index=False)
complete_breweries.head()

,name,brewery_type,address_1,city,state_province,postal_code,longitude,latitude,state,zip_code,brewery_id
0,(405) Brewing Co,micro,1716 Topeka St,Norman,Oklahoma,73069-8224,-97.468182,35.257389,Oklahoma,73069,1
1,10 Barrel Brewing Co,large,1501 E St,San Diego,California,92101-6618,-117.129593,32.714813,California,92101,3
2,10 Barrel Brewing Co,large,62970 18th St,Bend,Oregon,97701-9847,-121.281706,44.086835,Oregon,97701,4
3,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,Bend,Oregon,97703-2465,-121.328802,44.057565,Oregon,97703,5
4,10 Barrel Brewing Co,large,1411 NW Flanders St,Portland,Oregon,97209-2620,-122.685506,45.525979,Oregon,97209,6


In [16]:
# Loop through rows of completed brewery data to get final row count to confirm succsefull combination 
rowcount  = 0
#iterating through the whole file
for row in open("csv building blocks/breweries_complete.csv"):
  rowcount+= 1
 #printing the result
print("Number of rows:-", rowcount)

Number of rows:- 5545


In [17]:
# Base URL for census
base_url = 'https://api.census.gov/data/2021/acs/acs1/profile?'

# Define the parameter
# Parameters google sheet link https://docs.google.com/spreadsheets/d/1Hm1cXEWH2ccg9TD9DUkOKDIaYO88VxDwFeTTsmGfNik/edit#gid=0

parameter_list_final = ['DP02_0001E','DP02_0006E','DP02_0010E','DP02_0025E','DP02_0025PE','DP02_0031E','DP02_0031PE','DP02_0067E','DP02_0067PE','DP02_0068E',
'DP02_0068PE','DP03_0001PE','DP03_0001E','DP03_0051E','DP03_0062E','DP03_0063E','DP03_0075E','DP03_0086E','DP03_0087E','DP03_0088E','DP03_0089E',
'DP03_0090E','DP03_0091E','DP03_0119E','DP03_0119PE','DP03_0128E','DP03_0128PE','DP04_0001E','DP04_0001PE','DP04_0006E','DP04_0006PE','DP05_0001E',
'P05_0001PE','DP05_0022E','DP05_0022PE','DP05_0008E','DP05_0008PE','DP05_0026E','DP05_0026PE','DP05_0027E','DP05_0027PE','DP03_0089E']

#'P05_0001PE'
parameter_list_test = ['DP02_0001E','DP02_0006E','DP02_0010E','DP02_0025E','DP02_0025PE','DP02_0031E','DP02_0031PE','DP02_0067E','DP02_0067PE','DP02_0068E',
'DP02_0068PE','DP03_0001PE','DP03_0001E','DP03_0051E','DP03_0062E','DP03_0063E','DP03_0075E','DP03_0086E','DP03_0087E','DP03_0088E','DP03_0089E'
,'DP03_0090E','DP03_0091E','DP03_0119E','DP03_0119PE','DP03_0128E','DP03_0128PE','DP04_0001E','DP04_0001PE','DP04_0006E','DP04_0006PE','DP05_0001E',
'DP05_0022E','DP05_0022PE','DP05_0008E','DP05_0008PE','DP05_0026E','DP05_0026PE','DP05_0027E','DP05_0027PE','DP03_0089E']

census_data = []

# Loop through parameter list
for parameter in parameter_list_test:
    query_url = base_url + "get=NAME," + parameter + "&for=place:*&in=state:*" + "&key=" + census_key 
    response = requests.get(query_url).json()
    census_data.append(response)

# Create initial dataframce
first = census_data[0]
code = census_data[0][0][1]
merged_df = pd.DataFrame(first, columns = ['City',code, 'State ID', 'City ID'])

# Append multiple dataframes together (not finished yet)
for x in census_data:
    new_df = pd.DataFrame(x, columns = ['City',x[0][1], 'State ID', 'City ID'])
    result = pd.concat([merged_df, new_df], axis=1)

# Drop duplicate columns
df2 = result.T.drop_duplicates().T

# Drop 1st row
header_row = df2.iloc[0]
final_census_df = pd.DataFrame(df2.values[1:], columns=header_row)

#census_data
#new_df = pd.DataFrame(x, columns = ['City',x[0][1], 'State ID', 'City ID'])
census_data
#final_census_df
d_data = census_data[0]


#cols_1 = 'DP05_0022PE'###
cols = d_data[0] # list(zip(row for row in d_data[1:]))
rows = [dict(zip(cols, row)) for row in d_data[1:]]
rows
df = pd.DataFrame(rows)
df = df[['NAME','state','place',cols[1]]]
census_data
for d in census_data:

    df[d[0][1]] = [row[1] for row in d[1:]]

df.to_csv("csv building blocks/census_codes.csv", index=False)

census_final = df.rename(columns={'DP02_0001E':'households_total',
'DP02_0006E':'households_total_male',
'DP02_0010E':'households_total_female',
'DP02_0025E':'marital_status__male_15_over',
'DP02_0025PE':'marital_status__male_15_over_percent',
'DP02_0031E':'marital_status__female_15_over',
'DP02_0031PE':'marital_status__female_15_over_percent',
'DP02_0067E':'edu_high_school_over',
'DP02_0067PE':'edu_high_school_over',
'DP02_0068E':'edu_bach_degree_over',
'DP02_0068PE':'edu_bach_degree_over_percent',
'DP03_0001PE':'employment_status_percent',
'DP03_0001E':'employment_status',
'DP03_0051E':'income_household',
'DP03_0062E':'income_household_median',
'DP03_0063E':'income_household_mean',
'DP03_0075E':'income_family',
'DP03_0086E':'income_family_median',
'DP03_0087E':'income_family_mean',
'DP03_0088E':'income_per_capita',
'DP03_0089E':'income_nonfamily',
'DP03_0090E':'income_nonfamily_median',
'DP03_0091E':'income_nonfamily_mean',
'DP03_0119E':'income_below_poverty_family',
'DP03_0119PE':'income_below_poverty_percent_family',
'DP03_0128E':'income_below_poverty',
'DP03_0128PE':'income_below_poverty_percent',
'DP04_0001E':'housing_occupancy',
'DP04_0001PE':'housing_occupancy_percent',
'DP04_0006E':'housing_in_structure',
'DP04_0006PE':'housing_in_structure_percent',
'DP05_0001E':'population_total',
'DP05_0022E':'population_over_21',
'DP05_0022PE':'population_over_21_percent',
'DP05_0008E':'population_15_to_19',
'DP05_0008PE':'population_15_to_19_percent',
'DP05_0026E':'population_male_over_18',
'DP05_0026PE':'population_male_over_18_percent',
'DP05_0027E':'population_female_over_18',
'DP05_0027PE':'population_female_over_18_percent'})


In [76]:
#
census_final.to_csv("csv building blocks/census_names.csv", index=False)

In [50]:
# Save final_census to csv file
#final_census_df.to_csv('final_census.csv', index=False)
census_final.to_csv("csv building blocks/final_census_nl.csv", index_label="City_ID")

In [40]:
# Dependencies and Setup
#import matplotlib.pyplot as plt
#import pandas as pd
#import scipy.stats as st

# Study data files
brewery_last_path = "csv building blocks/breweries_missing_completed.csv"
census_last_path = "csv building blocks/final_census_nl.csv"

# Read the mouse data and the study results
brewery_last = pd.read_csv(brewery_last_path)
census_last = pd.read_csv(census_last_path, index_col="City_ID")

In [41]:
# Display dataframe 1
#rename city column to City
brewery_last = brewery_last.rename(columns={"city":"City"})
brewery_last.head()

,name,brewery_type,address_1,City,state_province,postal_code,longitude,latitude,state,zip_code,brewery_id
0,(512) Brewing Co,micro,407 Radam Ln Ste F200,Austin,Texas,78745-1197,NaN,NaN,Texas,78745,2
1,12 Gates Brewing Company,brewpub,80 Earhart Dr Ste 20,Williamsville,New York,14221-7804,NaN,NaN,New York,14221,19
2,12 West Brewing Company,micro,3000 E Ray Rd Bldg 6,Gilbert,Arizona,85296-7832,NaN,NaN,Arizona,85296,20
3,12welve Eyes Brewing,micro,141 E 4th St Ste LL2,Saint Paul,Minnesota,55101-1639,NaN,NaN,Minnesota,55101,25
4,13 Stripes Brewery,brewpub,"250 Mill St, Suite PW3101",Taylors,South Carolina,29687,NaN,NaN,South Carolina,29687,27


In [49]:
# Display dataframe 2
census_last.head()

,state,place,households_total,households_total_male,households_total_female,marital_status__male_15_over,marital_status__male_15_over_percent,marital_status__female_15_over,marital_status__female_15_over_percent,edu_high_school_over,...,population_over_21,population_over_21_percent,population_15_to_19,population_15_to_19_percent,population_male_over_18,population_male_over_18_percent,population_female_over_18,population_female_over_18_percent,City,State
City_ID,,,,,,,,,,,,,,,,,,,,,
0,Missouri,54074,34412.0,4333.0,7513.0,34707.0,34707.0,39512.0,39512.0,60074.0,...,66311,70.8,7222,7.7,32890,47.2,36774,52.8,O'Fallon,Missouri
1,Missouri,65000,139736.0,37975.0,57478.0,118761.0,118761.0,128187.0,128187.0,191070.0,...,228212,77.8,14865,5.1,114730,47.9,124557,52.1,St. Louis,Missouri
2,New Jersey,56550,20446.0,2636.0,7469.0,24484.0,24484.0,27650.0,27650.0,25164.0,...,44179,63.4,6414,9.2,22608,46.9,25559,53.1,Passaic,New Jersey
3,New Hampshire,50260,36986.0,8301.0,8689.0,38993.0,38993.0,37580.0,37580.0,60651.0,...,70673,77.6,5215,5.7,37699,51.2,35930,48.8,Nashua,New Hampshire
4,Minnesota,54880,49984.0,8903.0,13809.0,46609.0,46609.0,51593.0,51593.0,77451.0,...,88939,73.2,8678,7.1,44572,47.7,48846,52.3,Rochester,Minnesota


In [45]:
# split NAME into two columns where (,) 
new = census_last["NAME"].str.split(",", n = 1, expand = True)
#rename column 0 to "City"
new.rename(columns={0:'City'}, inplace=True)
#rename column 1 to "State"
new.rename(columns={1:'State'}, inplace=True)
#replace column with the new columns
census_last['City'] = new['City']
census_last['State'] = new['State']
#delete space after the City name in each row
census_last['City'] = census_last['City'].str[:-1]

#drop column NAME
census_last.drop(columns =["NAME"], inplace = True)

#drop last 4 charasters and leading space form the"City" column
census_last['City'] = census_last['City'].str[:-4]

census_last.head()

,state,place,households_total,households_total_male,households_total_female,marital_status__male_15_over,marital_status__male_15_over_percent,marital_status__female_15_over,marital_status__female_15_over_percent,edu_high_school_over,...,population_over_21,population_over_21_percent,population_15_to_19,population_15_to_19_percent,population_male_over_18,population_male_over_18_percent,population_female_over_18,population_female_over_18_percent,City,State
City_ID,,,,,,,,,,,,,,,,,,,,,
0,Missouri,54074,34412.0,4333.0,7513.0,34707.0,34707.0,39512.0,39512.0,60074.0,...,66311,70.8,7222,7.7,32890,47.2,36774,52.8,O'Fallon,Missouri
1,Missouri,65000,139736.0,37975.0,57478.0,118761.0,118761.0,128187.0,128187.0,191070.0,...,228212,77.8,14865,5.1,114730,47.9,124557,52.1,St. Louis,Missouri
2,New Jersey,56550,20446.0,2636.0,7469.0,24484.0,24484.0,27650.0,27650.0,25164.0,...,44179,63.4,6414,9.2,22608,46.9,25559,53.1,Passaic,New Jersey
3,New Hampshire,50260,36986.0,8301.0,8689.0,38993.0,38993.0,37580.0,37580.0,60651.0,...,70673,77.6,5215,5.7,37699,51.2,35930,48.8,Nashua,New Hampshire
4,Minnesota,54880,49984.0,8903.0,13809.0,46609.0,46609.0,51593.0,51593.0,77451.0,...,88939,73.2,8678,7.1,44572,47.7,48846,52.3,Rochester,Minnesota


In [56]:

#merge two dataframes by City column 
merged_df = pd.merge(brewery_last, census_last, on="City", how="left")

merged_df.head(30)
#drop merged df into csv file
merged_df.to_csv("csv building blocks/merged_df.csv", index=False)
merged_df

,name,brewery_type,address_1,City,state_province,postal_code,longitude,latitude,state_x,zip_code,...,population_total,population_over_21,population_over_21_percent,population_15_to_19,population_15_to_19_percent,population_male_over_18,population_male_over_18_percent,population_female_over_18,population_female_over_18_percent,State
0,(512) Brewing Co,micro,407 Radam Ln Ste F200,Austin,Texas,78745-1197,NaN,NaN,Texas,78745,...,964000.0,742813.0,77.1,55131.0,5.7,397010.0,50.8,384282.0,49.2,Texas
1,12 Gates Brewing Company,brewpub,80 Earhart Dr Ste 20,Williamsville,New York,14221-7804,NaN,NaN,New York,14221,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12 West Brewing Company,micro,3000 E Ray Rd Bldg 6,Gilbert,Arizona,85296-7832,NaN,NaN,Arizona,85296,...,273138.0,182419.0,66.8,21771.0,8.0,95833.0,49.9,96199.0,50.1,Arizona
3,12welve Eyes Brewing,micro,141 E 4th St Ste LL2,Saint Paul,Minnesota,55101-1639,NaN,NaN,Minnesota,55101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13 Stripes Brewery,brewpub,"250 Mill St, Suite PW3101",Taylors,South Carolina,29687,NaN,NaN,South Carolina,29687,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2513,Zebulon Artisan Ales,micro,8 Merchents Alley,Weaverville,North Carolina,28787-8860,NaN,NaN,North Carolina,28787,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2514,Zephyr Brewing Co,micro,1609 E 58th Ave Ste G,Denver,Colorado,80216-1528,NaN,NaN,Colorado,80216,...,711463.0,558313.0,78.5,35211.0,4.9,291170.0,50.3,287833.0,49.7,Colorado
2515,Zero Gravity Craft Brewery,micro,716 Pinte Street,Burlington,Vermont,05401,NaN,NaN,Vermont,05401,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2516,Zum Bier,micro,3232 E Monroe St,Waukegan,Illinois,60085,NaN,NaN,Illinois,60085,...,85791.0,65181.0,76.0,6012.0,7.0,35776.0,51.9,33194.0,48.1,Illinois


In [60]:
# Delete Rows by Checking Conditions

dfresult = merged_df.dropna(thresh=10)
dfresult

,name,brewery_type,address_1,City,state_province,postal_code,longitude,latitude,state_x,zip_code,...,population_total,population_over_21,population_over_21_percent,population_15_to_19,population_15_to_19_percent,population_male_over_18,population_male_over_18_percent,population_female_over_18,population_female_over_18_percent,State
0,(512) Brewing Co,micro,407 Radam Ln Ste F200,Austin,Texas,78745-1197,NaN,NaN,Texas,78745,...,964000.0,742813.0,77.1,55131.0,5.7,397010.0,50.8,384282.0,49.2,Texas
2,12 West Brewing Company,micro,3000 E Ray Rd Bldg 6,Gilbert,Arizona,85296-7832,NaN,NaN,Arizona,85296,...,273138.0,182419.0,66.8,21771.0,8.0,95833.0,49.9,96199.0,50.1,Arizona
5,1323 R & D,micro,1323 Capital Blvd 1323 R and D,Raleigh,North Carolina,27603-1117,NaN,NaN,North Carolina,27603,...,469502.0,352911.0,75.2,30428.0,6.5,179389.0,48.0,194590.0,52.0,North Carolina
13,212 Brewing Company,contract,21476 Route 23,Davenport,New York,13750,NaN,NaN,New York,13750,...,101010.0,76954.0,76.2,6022.0,6.0,39269.0,48.6,41612.0,51.4,Iowa
16,28th State Brewing,planning,NaN,Houston,Texas,77095-6933,NaN,NaN,Texas,77095,...,2287047.0,1664835.0,72.8,152355.0,6.7,864677.0,49.3,889900.0,50.7,Texas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,Young Veterans Brewing Company,micro,2505 Horse Pasture Rd Ste 104,Virginia Beach,Virginia,23453-2988,NaN,NaN,Virginia,23453,...,457672.0,339766.0,74.2,27482.0,6.0,173658.0,48.7,183233.0,51.3,Virginia
2512,Zambaldi Beer,contract,NaN,Green Bay,Wisconsin,54305-0246,NaN,NaN,Wisconsin,54305,...,107020.0,80914.0,75.6,6560.0,6.1,40906.0,48.2,43940.0,51.8,Wisconsin
2514,Zephyr Brewing Co,micro,1609 E 58th Ave Ste G,Denver,Colorado,80216-1528,NaN,NaN,Colorado,80216,...,711463.0,558313.0,78.5,35211.0,4.9,291170.0,50.3,287833.0,49.7,Colorado
2516,Zum Bier,micro,3232 E Monroe St,Waukegan,Illinois,60085,NaN,NaN,Illinois,60085,...,85791.0,65181.0,76.0,6012.0,7.0,35776.0,51.9,33194.0,48.1,Illinois


In [61]:
len(dfresult.index)

1023

In [ ]:
dfresult.to_csv("csv building blocks/merged_df_cleaned.csv", index=False)

In [ ]:
#merge back missing info into the original breweries csv:
#you might need to first drop the longitude and latitude empty rows from the original dataframe clean_breweries_df and then add 
#the csv_breweries_missing_completed_df to the clean_breweries_df


#create a new clean dataframe that we can use for analysis by dropping remaining missing lat&long


#load the the final "clean dataframe" into the csv file

In [ ]:
#extract a separate data set for "closed" breweries for separate analysis - VERY FEW btw 162 - and create a new dataframe for that one

#extract a separate data set for "planning" breweries for separate analysis - should be about 704 - and create a new dataframe for that one

#QUESTION FOR THE TEAM: Do we want to focus on "micro" and "brewpubs" only? if so, delete rows with all other brewery types


In [ ]:
# create a summary dataframe with number of breweries by zip_code - listing City, State, and Zip Code, and brewery count  

# load additional information regrdidng the zipcode/each brewery cluster data from https://uszipcode.readthedocs.io/
# review documentation by following the link to see how the information populates and if useful for our analysis
# Some of the data to consider --> household_income; median_household_income population, population density, families_vs_singles, 
# median_home_value, educational_attainment_for_population_25_and_over 




In [ ]:
#explore Census API Community Data API for any additional information that might be useful for our analysis - gender, race etc.
#explore articles for inspiration on what other data might be useful for our analysis
# https://www.census.gov/library/visualizations/interactive/breweries.html  --> Article about breweries for the Census 
#https://cbb.census.gov/cbb/#industry0=312120&geoId=17031&geoType=county&view=report&reportType=summary --> Brewery Busisness in Illinois 


#load relevant data into the zipcode summary table 